##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Crie um modelo linear com Estimadores

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/linear"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

> Aviso: os Estimadores não são recomendados para novos códigos.  Os Estimadores executam código `v1.Session`, que é mais difícil de escrever corretamente e pode se comportar de forma inesperada, ainda mais quando usado em conjunto com código do TF 2. Os Estimadores são abarcados pelas [garantias de compatibilidade](https://tensorflow.org/guide/versions), mas não recebem mais correções, exceto para vulnerabilidades de segurança. Confira mais detalhes no [guia de migração](https://tensorflow.org/guide/migrate).

## Visão geral

Este tutorial de ponta a ponta treina um modelo de regressão logística usando a API `tf.estimator`. Esse modelo é geralmente usado como linha de base para outros algoritmos mais complexos.

Observação: um exemplo de regressão logística do Keras está [disponível](https://tensorflow.org/guide/migrate/tutorials/keras/regression) e é recomendado neste tutorial.


## Configuração

In [ ]:
!pip install sklearn


In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

## Carregue o dataset Titanic

Você usará o dataset Titanic com o objetivo (bastante mórbido) de prever a sobrevivência dos passageiros, considerando características como gênero, idade, classe, etc.

In [ ]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

## Explore os dados

O dataset contém as seguintes características:

In [ ]:
dftrain.head()

In [ ]:
dftrain.describe()

Há 627 e 264 exemplos nos datasets de treinamento e avaliação, respectivamente.

In [ ]:
dftrain.shape[0], dfeval.shape[0]

A maioria dos passageiros tem cerca de 20 e 30 anos.

In [ ]:
dftrain.age.hist(bins=20)

Há aproximadamente duas vezes mais homens que mulheres a bordo.

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')

A maioria dos passageiros está na "terceira" classe.

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')

As mulheres têm uma chance muito mais alta de sobrevivência em comparação com os homens. Isso é claramente uma característica preditiva para o modelo.

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

## Engenharia de características para o modelo

> Aviso: o módulo tf.feature_columns descrito neste tutorial não é recomendado para novos códigos. As camadas de pré-processamento do Keras abrangem essa funcionalidade. Para instruções de como migrar, confira o [Guia de migração das colunas de características](https://www.tensorflow.org/guide/migrate/migrating_feature_columns). O módulo tf.feature_columns foi feito para ser usado com Estimadores TF1. Ele é abarcado pelas [garantias de compatibilidade](https://tensorflow.org/guide/versions), mas não recebe mais correções, exceto para vulnerabilidades de segurança.

Os Estimadores usam um sistema chamado de [colunas de características](https://www.tensorflow.org/tutorials/structured_data/feature_columns) para descrever como o modelo deve interpretar cada uma das características de entrada brutas. Um Estimador espera um vetor de entradas numéricas, e as *colunas de características* descrevem como o modelo deve converter cada característica.

A seleção e a criação do conjunto ideal de colunas de características são fundamentais para o aprendizado eficaz de um modelo. Uma coluna de características pode ser uma das entradas brutas no `dict` das características originais (uma *coluna de características base*) ou qualquer coluna nova criada usando transformações definidas em uma ou mais colunas base (uma *coluna de características derivadas*).

O estimador linear usa tanto características numéricas quanto categóricas. As colunas de características funcionam com todos os estimadores do TensorFlow e o objetivo delas é definir as características usadas para a modelagem. Além disso, elas oferecem alguns recursos de engenharia de características para one-hot-encoding, normalização e bucketization.

### Colunas de características base

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

A `input_function` especifica como os dados são convertidos em um `tf.data.Dataset` que alimenta o pipeline de entrada como um streaming. `tf.data.Dataset` aceita várias origens, como um dataframe, um arquivo formatado em CSV e muito mais.

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Você pode inspecionar o dataset:

In [ ]:
ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

Você também pode inspecionar o resultado de uma determinada coluna de características usando a camada `tf.keras.layers.DenseFeatures`:

In [ ]:
age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()

`DenseFeatures` só aceita tensores densos. Para inspecionar uma coluna categórica, você precisa primeiro transformá-la em uma coluna indicadora:

In [ ]:
gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

Depois de adicionar todos os recursos base ao modelo, vamos treiná-lo. O treinamento de um modelo é apenas um único comando usando a API `tf.estimator`:

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

### Colunas de características derivadas

Agora, você alcançou uma exatidão de 75%. Usar cada coluna de características base separadamente pode não ser o suficiente para explicar os dados. Por exemplo, a correlação entre a idade e o rótulo pode ser diferente para cada gênero. Portanto, se você aprender um único peso de modelo para `gender="Male"` e `gender="Female"`, não vai capturar todas as combinações de idade e gênero (por exemplo, distinguindo entre `gender="Male"` E `age="30"` E `gender="Male"` E `age="40"`).

Para aprender as diferenças entre as combinações de características, você pode adicionar *colunas cruzadas de características* ao modelo (você também pode fazer o bucketization da coluna de idade antes da coluna cruzada):

In [ ]:
age_x_gender = tf.feature_column.crossed_column(['age', 'sex'], hash_bucket_size=100)

Depois de adicionar as características de combinação ao modelo, vamos treiná-lo novamente:

In [ ]:
derived_feature_columns = [age_x_gender]
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

Ele agora alcança uma exatidão de 77,6%, que é um pouco melhor do que somente com o treinamento de características base. Você pode tentar usar mais características e transformações para ver se consegue melhorá-lo!

Agora, você pode usar o modelo de treinamento para fazer previsões sobre um passageiro a partir do dataset de avaliação. Os modelos do TensorFlow são otimizados para fazer previsões em um lote, ou coleção, de exemplos ao mesmo tempo. Antes, o `eval_input_fn` era definido usando todo o dataset de avaliação.

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

Por fim, confira a característica de operação do receptor (ROC) dos resultados, que dará uma ideia melhor do trade-off entre a taxa de positivo verdadeiro e falso positivo.

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)